# P1 (and B1) leakage correction

In nuclear reactor analysis and core design, a traditional two-step (or multi-step) approach is typically implemented. First, neutron transport is solved using high-fidelity methods in smaller core domains (e.g., at the level of pin cells or fuel assemblies with reflective boundaries—often referred to as lattice-level calculations) to produce homogenized (spatially) and condensed (over energy) cross-section data for full-core simulators. However, the infinite systems modeled at the lattice level are typically far from criticality because, due to power shaping, a reactor core contains both more and less reactive fuel assemblies.

Therefore, after producing the homogenized cross-sections, a leakage correction is often applied to condense the cross-sections with a critical spectrum. The justification for this correction is that most nuclear reactors, under normal operation, run at or near a critical state. As a result, fuel elements should experience a critical spectrum, necessitating a correction to the spectrum of the infinite and non-critical system before performing the condensation.

The need for a criticality spectrum calculation, or the method to be performed is sometimes debated (see for example [a paper from K. Smith](https://www.sciencedirect.com/science/article/pii/S0149197017301609)), nevertheless there is pedagogical value to study and implement such methods.

We will implement the P1 method (however the implementation is essentially the same for the B1 method). The derivation of the equations can be found in several text books (see for example [Duderstadt and Hamilton](https://deepblue.lib.umich.edu/handle/2027.42/89079) p355), and is omitted here. 

In this notebook, we follow the steps described by Stamm'ler and Abbate in Methods of Steady-state Reactor Physics in Nuclear Design (Chapter X), since it is so detailed and well described that the implementation becomes almost too easy.

As said, the derivation is omitted here (mostly because it is hard to compete with the above mentioned excellent books) and we just introduce the P1 equations after the angular flux is separated into a spatial and an angle-energy mode, and the latter is Legendre expended up to the second term:

$$\Sigma (E)\psi(E)\pm iBJ(E)=\int \Sigma_s0(E'\rightarrow E)\psi(E')dE'+\chi(E)$$

$$\pm iB\psi(E)+3\Sigma(E)J(E)=3\int\Sigma_s1(E'\rightarrow E)J(E')dE'$$

The B1 equations only slightly differ by introducing an $\alpha(B,E)$ multiplier in the second equation's $3\Sigma(E)J(E)$ term. The spatial solution is assumed to have $\exp(\pm iBz)$ shape, hence the imaginary number appears, but fear it not.

To solve this system, first we convert it into multigroup formalism (simply, the energy dependence becomes group dependence and integrals become sums): 

$$\Sigma_g\psi_g-\sum_h\Sigma_{s0,g\leftarrow h}\psi_h\pm iBJ_g=\chi_g$$

$$3\Sigma_gJ_g-3\sum_h\Sigma_{s1,g\leftarrow h}J_h=\mp iB\psi_g$$

One can cast these into a matrix equations

$$\bar J = \mp iB\mathbf{D}\bar\psi \quad \text{where} \quad D_{gh}^{-1}=3\Sigma_g\delta_{gh}-3\Sigma_{s1,g\leftarrow h}$$

$$\mathbf{A}\bar\psi=\bar\chi \quad \text{where} \quad A_{gh}=\Sigma_g\delta_{gh}-\Sigma_{s0,g\leftarrow h}+B^2D_{gh}$$

and the multiplication factor of the system can be obtained with

$k=\sum_g\nu_g\Sigma_{f,g}\psi_g$

Hence, our task is to build the matrix $\mathbf{D}^{-1}$, invert it, then build matrix $\mathbf{A}$ and solve for $\bar\psi$, then evaluate the multiplication factor $k$. With that we would obtain the infinite multiplication factor, but we are still not done. We need to iterate the spectrum until we find the critical spectrum. For the iteration we can follow the recipe from Stamm'ler and Abbate:

1. Set $B^2$=0 (ie. the zero current, or no leakage case). Solve for the flux and find $k_\infty$.
2. Set $B^2$=0.001 if $k_\infty>1.0$, and -0.001 otherwise. Find, the multiplication factor $k_1$.
3. Perform an interpolation with the two evaluations of $B^2(k)$ in order to find $B^2(k=1)$.
   - Evaluate the slope as $m=\frac{B^2(k_1)}{\frac{1}{k_1}-\frac{1}{k_\infty}}$
4. Estimate $B^2(k=1)=B^2(k_1)+m\cdot (1-\frac{1}{k_1})$ 
5. Solve for the flux and find $k_2$ (or $k_i$ in subsequent iterations)
6. Repeat step 4 and 5 while $|k_i-1|>5\cdot 10^{-6}$. Note, the slope should not be reevaluated.

So, let's get to it! We will use `numpy.linalg` for the matrix operations.

First, we load the data, which is prepared in the accompanying `prepareData.ipynb` notebook.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

with open('homogenizedXS.json') as f:
    gxs = json.load(f)

NG=gxs['NG']
Sig=np.array(gxs['Sig'])
SigS0=np.array(gxs['SigS0']).reshape((NG,NG))
SigS1=np.array(gxs['SigS1']).reshape((NG,NG))
nuSigf=np.array(gxs['nuSigf'])
Chi=np.array(gxs['Chi'])
micro=np.array(gxs['micro'])
b1Flx=np.array(gxs['b1Flx'])   #for benchmarking
infFlx=np.array(gxs['infFlx']) #for benchmarking

Then, implement functions to construct the matrices.

In [ ]:
kronecker = lambda i,j : 0 if i!=j else 1
eps=5e-6
def makeDinv():
    #function to create the Dinv matrix. We can use the necessary data from globals.
    return Dinv

def makeA():
    #function to create the A matrix. We can use the necessary data from globals.
    return A

def getAlpha(B2,Sigma):
    #The eager ones can implement the B1 method
    pass
    

Finally perform the recipe given by Stamm'ler and Abbate.

In [ ]:

#make Dinv and invert with np.linalg.inv.
#One might also visualize this matrix, since this the diffiusion coeff in the multi-group generalization of Fick's law


#Initial guess: zero current
B2=0.0
#create matrix A, solve for psi with np.linalg.solve
#keep this as the infinite flux, so we can later evaluate the ratio of the infinite and critical spectrum

#evaluate k-inf

print(kinf)

#New guess for B2, Solve again for psi and k1
B2 = 0.001 if kinf>1 else -0.001

#evaluate the slope

#iterate while we are not close to 1
while abs(k-1)>eps:
    #solve for psi and k
    
    print(k)



Let us visualize the ratio of the criticality (your final flux vector) and infinite spectrum (your first flux vector corresponding to $B^2=0$). Which parts increased? What is the consequence of that in terms of leakage? 

We use the results from Serpent2 for benchmarking. Note, that in order to make your flux ratio comparable, you need to normalize it (ie. divide with the sum).

In [ ]:

    plt.figure()
    plt.title('Fundamental mode criticality spectrum calculation')
    #your results
    plt.semilogx(micro[:-1],b1Flx/infFlx,'m',label='Serpent2 implementation') #slight laziness and cheating with the energy variable
    plt.xlabel('Energy (MeV)')
    plt.ylabel('Critical spectrum / Infinite spectrum (-)')
    plt.legend()
    plt.tight_layout()
    plt.show()


## Possible further exercises

- Implement the B1 correction.
- Obtain the group-wise diffusion coefficients ($D_g=\frac{J_g}{|B|\psi_g}$)
- Create data for varying boron content in the coolant (if you do not have access to Serpent2, you can use for example openMC), observe the impacts of the P1 and B1 correction on the diffusion coefficient. For this you can perform the cross section condensation to two groups first, and then investigate the fast and thermal group constants.

